# Reinforcement Learning for Cart Pole

### 1) Import Dependencies

In [ ]:
!pip install stable_baselines3[extra]

In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### 2) Load Environment

In [2]:
log_path = os.path.join('Training', 'Logs')

In [3]:
# Mapping to the pre-installed open AI gym Environment
env = gym.make("CartPole-v0")
env = DummyVecEnv([lambda : env])

### 3) Train an RL Model

In [4]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [5]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 375  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 515          |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0094319265 |
|    clip_fraction        | 0.117        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | -0.00114     |
|    learning_rate        | 0.0003       |
|    loss                 | 6.61         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0174      |
|    value_loss           | 52.9         |
----------------------------

### 4) Evaluation

In [27]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
#env.close()

C:\Users\Ebru\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

### 5) Test Model

In [11]:
obs = env.reset()
score = 0
while True:
    action, _ = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    score += rewards
    if done: 
        print('info: ', info)
        print('\nScore:{}'.format(score))
        break
env.close()

info:  [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.16600522,  0.00799892,  0.13289626, -0.40128252], dtype=float32)}]

Score:[200.]


### 6) Save the Model

In [12]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')
model.save(PPO_path)
#del model
#model = PPO.load('PPO_model', env=env)

### 7) Tensorboard

In [13]:
training_log_path = os.path.join(log_path, 'PPO_2')
!tensorboard --logdir={training_log_path}

^C


### 8) Callback

In [14]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [26]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=195, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=os.path.join('Training', 'Saved Models'), 
                             verbose=1)

In [27]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 1585 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1039        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009221504 |
|    clip_fraction        | 0.111       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00607     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.92        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0177     |
|    value_loss           | 54.1        |
----------------------------

In [28]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [29]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [30]:
env.close()